In [2]:
# load MNE, NeuroKit and other useful packages
import math
import numpy as np
import pandas as pd
import neurokit2 as nk
import os
import mne
import csv
from scipy.integrate import cumtrapz
from scipy.io import savemat
import matplotlib.pyplot as plt

### Function to gain ECG_TS

In [3]:
# This function extracts ECG power time series during movie viewing.
#
# Output includes:
# - IBI (Inter-Beat Interval) time series
# - LF-HRV (Low-Frequency Heart Rate Variability) time series
# - HF-HRV (High-Frequency Heart Rate Variability) time series

def movie_ECG_TS(id, age):
    id = id
    age = age
    
# specify the save path
    savepath = '/brain-heart interplay/movie_ECG_TS/'
    savename = str(id) + '_movie_ECG_TS.mat'
    savefile = os.path.join(savepath, savename)
# Check if the file exists and exit the function if it does
    if os.path.exists(savefile):
        print(f"File {savename} already exists. Exiting function.")
        return

# EEG folder    
    data_path = '/raw_eeg'
    movie_filename = str(id) + '_movie.vhdr'
    movie_path = os.path.join(data_path, str(id), movie_filename)
# read in data, ECG
    movie_EEG = mne.io.read_raw_brainvision(movie_path)
    
## event marker
# S1-2 fear S3-4 neutral S5-6 happy
# in seconds
    fear_onset = [ann['onset'] for i, ann in enumerate(movie_EEG._annotations) if ann['description'] == 'Stimulus/S  1' ]
    neutral_onset = [ann['onset'] for i, ann in enumerate(movie_EEG._annotations) if ann['description'] == 'Stimulus/S  3' ]
    happy_onset = [ann['onset'] for i, ann in enumerate(movie_EEG._annotations) if ann['description'] == 'Stimulus/S  5' ]
## ECG channel is '65'
# drop out the first 2 seconds just like the EEG prepocessing
# e.g., ECG_happy is a tuple of (data_array, times_array), times in seconds
    ECG_happy_raw = movie_EEG['65', (happy_onset[0]+2)*500:(happy_onset[0]+180)*500] # sample rate is 500 Hz
    ECG_fear_raw = movie_EEG['65', (fear_onset[0]+2)*500:(fear_onset[0]+180)*500]
    ECG_neutral_raw = movie_EEG['65', (neutral_onset[0]+2)*500:(neutral_onset[0]+180)*500]
# transform to shape a vector to input 
    happy_ECGdata = ECG_happy_raw[0].reshape(-1,1)
    fear_ECGdata = ECG_fear_raw[0].reshape(-1,1)
    neutral_ECGdata = ECG_neutral_raw[0].reshape(-1,1)
# find R peaks
    ECG_happy_nk, info_happy = nk.bio_process(ecg = happy_ECGdata, sampling_rate = 500)
    ECG_fear_nk, info_fear = nk.bio_process(ecg = fear_ECGdata, sampling_rate = 500)
    ECG_neutral_nk, info_neutral = nk.bio_process(ecg = neutral_ECGdata, sampling_rate = 500)
## calculate the ECG time series
# IBI series and resample at 4 Hz
    IBI_happy = nk.signal_period(peaks = info_happy['ECG_R_Peaks'], sampling_rate = 500, 
                            interpolation_method = 'signal_interpolate')
    IBI_time_ha = np.array(info_happy['ECG_R_Peaks'])/500
    IBI_happy_res = nk.intervals_process(IBI_happy*1000, intervals_time = IBI_time_ha, interpolate = True, interpolation_rate = 4)

    IBI_fear = nk.signal_period(peaks = info_fear['ECG_R_Peaks'], sampling_rate = 500, 
                            interpolation_method = 'signal_interpolate')
    IBI_time_fe = np.array(info_fear['ECG_R_Peaks'])/500
    IBI_fear_res = nk.intervals_process(IBI_fear*1000, intervals_time = IBI_time_fe, interpolate = True, interpolation_rate = 4)

    IBI_neutral = nk.signal_period(peaks = info_neutral['ECG_R_Peaks'], sampling_rate = 500, 
                            interpolation_method = 'signal_interpolate')
    IBI_time_ne = np.array(info_neutral['ECG_R_Peaks'])/500
    IBI_neutral_res = nk.intervals_process(IBI_neutral*1000, intervals_time = IBI_time_ne, interpolate = True, interpolation_rate = 4)
# HRV analysis--time frequency
# lf 0.04-0.24, hf 0.24-1.04 for child aged 5-10
    f_ha, t_ha, pwvd_ha = nk.signal_timefrequency(IBI_happy_res[0],
                                                        sampling_rate = 4,
                                                        min_frequency = 0.04,
                                                        max_frequency = 1.04,
                                                        method = "pwvd",
                                                        show = False)
    f_fe, t_fe, pwvd_fe = nk.signal_timefrequency(IBI_fear_res[0],
                                                        sampling_rate = 4,
                                                        min_frequency = 0.04,
                                                        max_frequency = 1.04,
                                                        method = "pwvd",
                                                        show = False)
    f_ne, t_ne, pwvd_ne = nk.signal_timefrequency(IBI_neutral_res[0],
                                                        sampling_rate = 4,
                                                        min_frequency = 0.04,
                                                        max_frequency = 1.04,
                                                        method = "pwvd",
                                                        show = False)
# ensure no neg value
    pwvd_ne[pwvd_ne < 0] = 0
    pwvd_ha[pwvd_ha < 0] = 0
    pwvd_fe[pwvd_fe < 0] = 0
    
# lf_time series, 1Hz
# lf frequency, sum up
    lf_ha = np.where((f_ha >= 0.04) & (f_ha <= 0.24))[0]
    lf_pwvd_ha = np.sum(pwvd_ha[lf_ha, :], axis=0)
# integrate power per seconds
    lf_pwvd_ha_1Hz = np.array([np.trapz(lf_pwvd_ha[i * 4:(i + 1) * 4], dx=0.25) 
                           for i in range(len(lf_pwvd_ha) // 4)])

    lf_fe = np.where((f_fe >= 0.04) & (f_fe <= 0.24))[0]
    lf_pwvd_fe = np.sum(pwvd_fe[lf_fe, :], axis=0)
# integrate power per seconds
    lf_pwvd_fe_1Hz = np.array([np.trapz(lf_pwvd_fe[i * 4:(i + 1) * 4], dx=0.25) 
                           for i in range(len(lf_pwvd_fe) // 4)])

    lf_ne = np.where((f_ne >= 0.04) & (f_ne <= 0.24))[0]
    lf_pwvd_ne = np.sum(pwvd_ne[lf_ne, :], axis=0)
# integrate power per seconds
    lf_pwvd_ne_1Hz = np.array([np.trapz(lf_pwvd_ne[i * 4:(i + 1) * 4], dx=0.25) 
                           for i in range(len(lf_pwvd_ne) // 4)])

# hf_time series, 1Hz
# hf frequency, sum up
    hf_ha = np.where((f_ha >= 0.24) & (f_ha <= 1.04))[0]
    hf_pwvd_ha = np.sum(pwvd_ha[hf_ha, :], axis=0)
# integrate power per seconds
    hf_pwvd_ha_1Hz = np.array([np.trapz(hf_pwvd_ha[i * 4:(i + 1) * 4], dx=0.25) 
                           for i in range(len(hf_pwvd_ha) // 4)])

    hf_fe = np.where((f_fe >= 0.24) & (f_fe <= 1.04))[0]
    hf_pwvd_fe = np.sum(pwvd_fe[hf_fe, :], axis=0)
# integrate power per seconds
    hf_pwvd_fe_1Hz = np.array([np.trapz(hf_pwvd_fe[i * 4:(i + 1) * 4], dx=0.25) 
                           for i in range(len(hf_pwvd_fe) // 4)])

    hf_ne = np.where((f_ne >= 0.24) & (f_ne <= 1.04))[0]
    hf_pwvd_ne = np.sum(pwvd_ne[hf_ne, :], axis=0)
# integrate power per seconds
    hf_pwvd_ne_1Hz = np.array([np.trapz(hf_pwvd_ne[i * 4:(i + 1) * 4], dx=0.25) 
                           for i in range(len(hf_pwvd_ne) // 4)])

## save data in .mat to be read in matlab
# Create a dictionary to store the data
    movie_ECG_TS = {
        'happy_lf': lf_pwvd_ha_1Hz,
        'happy_hf': hf_pwvd_ha_1Hz,
        'happy_RR':IBI_happy,
        'happy_RR_res':IBI_happy_res,
        'fear_lf': lf_pwvd_fe_1Hz,
        'fear_hf': hf_pwvd_fe_1Hz,
        'fear_RR':IBI_fear,
        'fear_RR_res':IBI_fear_res,
        'neutral_lf': lf_pwvd_ne_1Hz,
        'neutral_hf': hf_pwvd_ne_1Hz,
        'neutral_RR':IBI_neutral,
        'neutral_RR_res':IBI_neutral_res
    }

    savemat(savefile, movie_ECG_TS)

### Processing in loop

In [ ]:
# processing in loop
folderBase = ''
info_file = os.path.join(folderBase, 'child_info.csv')
info_id = pd.read_csv(info_file, delimiter=",") # with header

for i in range(0,len(info_id.ID),1):
    movie_ECG_TS(info_id.ID[i], info_id['Age_month'][i])